# Download Attachments to Excel

This notebook demonstrates how to download attachments from a Feature Layer in ArcGIS Online (AGOL) and save them to an Excel file. The Excel file will include each feature in the feature layer as a row and a column for each attachment embedded as an iframe linking to the locally downloaded attachments.

### Import Libraries

Import the necessary libraries for working with ArcGIS, handling data, and creating widgets.

```python

In [1]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
import getpass
import os
import shutil
import ipywidgets as widgets
from IPython.display import display, HTML

### AGOL Authentication
Prompt the user for their ArcGIS Online (AGOL) credentials and establish a connection to AGOL.

In [2]:
# Prompt user for AGOL credentials
username = input("Enter your AGOL username: ")
password = getpass.getpass("Enter your AGOL password: ")

# Connect to AGOL
gis = GIS("https://www.arcgis.com", username, password)


## Select Feature Layer
Provide the AGOL item ID, retrieve the item, and create a dropdown widget for selecting the desired layer.
Also, create a directory to save the attachments.

In [3]:

# Provide the AGOL item ID
item_id = "caaecc422e244897ad5a9d5e0ecf156b"

# Get the item
item = gis.content.get(item_id)

# Create a dropdown widget for selecting the layer
layer_options = [(layer.properties.name, i) for i, layer in enumerate(item.layers)]
layer_dropdown = widgets.Dropdown(
    options=layer_options,
    description='Select Layer:',
    disabled=False,
)

def on_layer_change(change):
    global feature_layer
    feature_layer = item.layers[change['new']]
    print(f"Selected layer: {feature_layer.properties.name}")

layer_dropdown.observe(on_layer_change, names='value')

# Display the dropdown widget
display(layer_dropdown)

# Create a directory to save attachments
output_dir = "attachments"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Dropdown(description='Select Layer:', options=(('admin_0', 0), ('extreme_events', 1)), value=0)

Selected layer: extreme_events


### Download Attachments
Define a class ArcGISAttachmentDownloader with methods to download attachments from the selected feature layer and save them to the specified directory.

In [4]:
class ArcGISAttachmentDownloader:
    '''
        A class to download attachments from an ArcGIS feature layer
        Attributes
        ----------
            feature_layer : FeatureLayer
            An ArcGIS feature layer object
            
            output_dir : str
            The directory to save the attachments
        Methods
        -------
            download_attachments()
            Download attachments from the feature layer to the output directory
            
            createExcel()
            Create an Excel file with the feature data and embedded images
    '''	
    # Function to download attachments
    def download_attachments(self, feature_layer, output_dir):

        features = feature_layer.query(where="1=1", out_fields="*").features

        #print the index of the feature
        # Initialize an empty list to store feature data
        data = []

        for feature in features[15:20]:
            object_id = feature.attributes['objectid']
            globalid = feature.attributes['globalid']
            attachments = feature_layer.attachments.get_list(object_id)
            #create a list of feature attributes
            feature_attributes = feature.attributes
        
            
            if attachments:
                for attachment in attachments:
                    attachment_id = attachment['id']
                    parentGlobalid = attachment['parentGlobalId']
                    attachment_name= attachment['name']
                    final_attachment_path = os.path.join(output_dir, f"{globalid}_{attachment_name}")
                
                    # Check if the attachment already exists
                    if not os.path.exists(final_attachment_path):
                        temp_dir = os.path.join(output_dir, f"temp_{object_id}")
                        
                        # Create a temporary directory to download the attachment
                        if not os.path.exists(temp_dir):
                            os.makedirs(temp_dir)
                        
                        # Download the attachment to the temporary directory
                        feature_layer.attachments.download(oid=object_id, attachment_id=attachment_id, save_path=temp_dir)
                        
                        # Move the attachment from the temporary directory to the output directory with the new name
                        temp_attachment_path = os.path.join(temp_dir, attachment_name)
                        shutil.move(temp_attachment_path, final_attachment_path)
                        
                        # Remove the temporary directory
                        shutil.rmtree(temp_dir)
                        
                        print(f"Downloaded {attachment_name} for feature {object_id} as {final_attachment_path}")
                    else:
                        print(f"Attachment {attachment_name} for feature {object_id} already exists as {final_attachment_path}")
                    
                    # Append the feature data to the list
                    feature_attributes.update({
                        'final_attachment_path': final_attachment_path,
                        "attachmentParentGlobalid": parentGlobalid
                    })
                    data.append(feature_attributes)
            else:
                # Append the feature data with no attachments
                feature_attributes.update({
                    'final_attachment_path': None,
                    'attachmentParentGlobalid': None
                })
                data.append(feature_attributes)
        return data
    
    def createExcel(self, data):
        # Create a DataFrame from the list
        df = pd.DataFrame(data)
        df.head()
        # Save the DataFrame to an Excel file
        output_excel_path = os.path.join(output_dir, 'features_with_attachments.xlsx')
        df.to_excel(output_excel_path, index=True, engine='xlsxwriter')

        # Load the workbook and worksheet to embed images
        with pd.ExcelWriter(output_excel_path, engine='xlsxwriter') as writer:
            df.to_excel(writer, index=False)
            workbook = writer.book
            worksheet = writer.sheets['Sheet1']

            # Widen the first column to make the caption clearer
            worksheet.set_column(0, 0, 30)
            worksheet.write(0, 0, "Embed images that scale to cell size")

            # Embed images in the final_attachment_path column
            for row_num, attachment_path in enumerate(df['final_attachment_path'], start=1):
                if attachment_path:
                    #adjust attachment path to be fully qualified path
                    attachment_path = os.path.abspath(attachment_path)
                    print(f"Embedding image {attachment_path} in row {row_num}")

                    # check for file extension embed images as jpg, jpeg, png using embed_image and hyperlink other file types
                    if attachment_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                        worksheet.set_row(row_num, 60)  # Adjust row height
                        worksheet.embed_image(row_num, df.columns.get_loc('final_attachment_path'), attachment_path, {'x_scale': 0.5, 'y_scale': 0.5})

                    else:
                        # Add a hyperlink to the file
                        worksheet.write_url(row_num, df.columns.get_loc('final_attachment_path'), f'external:{attachment_path}', string='Link to attachment')


        print(f"DataFrame saved as Excel file with embedded images at {output_excel_path}")

        return output_excel_path
    


### Execute Download and Save to Excel
Create an instance of ArcGISAttachmentDownloader, download the attachments, and save the data to an Excel file.

In [5]:
attachment_downloader = ArcGISAttachmentDownloader()
data = attachment_downloader.download_attachments(feature_layer, output_dir)
output_excel_path = attachment_downloader.createExcel(data)

Downloaded Screenshot_20240614-221010.jpg for feature 194 as attachments\3e0ac251-71a8-488f-8deb-bce2e3ea84e0_Screenshot_20240614-221010.jpg
Downloaded Screenshot_20240614-180234.jpg for feature 194 as attachments\3e0ac251-71a8-488f-8deb-bce2e3ea84e0_Screenshot_20240614-180234.jpg
Downloaded Screenshot_20240616-180857.jpg for feature 195 as attachments\13b05cad-9f1b-4ad0-8500-620fbbb1fb2a_Screenshot_20240616-180857.jpg
Downloaded Screenshot_20240615-110620.jpg for feature 195 as attachments\13b05cad-9f1b-4ad0-8500-620fbbb1fb2a_Screenshot_20240615-110620.jpg
Downloaded press_20220914.pdf for feature 211 as attachments\1a0ef617-0b09-46a5-8c73-f19f37415a4b_press_20220914.pdf
Downloaded press_20230928.pdf for feature 211 as attachments\1a0ef617-0b09-46a5-8c73-f19f37415a4b_press_20230928.pdf
Embedding image c:\Users\rami8629\OneDrive - Esri\Demos & Blogs\ArcGIS Resources\Code-Repos\ArcGIS_Code_Repo\src\scripts\Feature Service Management\attachments\3e0ac251-71a8-488f-8deb-bce2e3ea84e0_Scree